In [ ]:
import pandas as pd

df = pd.read_json("hf://datasets/MohammadOthman/mo-customer-support-tweets-945k/preprocessed_data.json")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


In [ ]:
df.head()

,input,output
0,is the worst customer service,I would love the chance to review the account ...
1,y’all lie about your “great” connection. 5 bar...,H there! we would definitely like to work with...
2,Since I signed up with you....Since day 1,We understand your concerns and we would like ...
3,You gonna magically change your connectivity f...,This is saddening to hear. Please shoot us a D...
4,"whenever I contact customer support, they tell...",Please send me a private message so that I can...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 945278 entries, 0 to 945277
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   input   945278 non-null  object
 1   output  945278 non-null  object
dtypes: object(2)
memory usage: 14.4+ MB


In [ ]:
import json

In [ ]:
def convert_to_token_format(user_input, assistant_output):
    return [
        {"token": "<|im_start|>"},
        f"system\nYou are a helpful customer support assistant.",
        {"token": "<|im_end|>"},
        "\n",
        {"token": "<|im_start|>"},
        f"user\n{user_input}",
        {"token": "<|im_end|>"},
        "\n",
        {"token": "<|im_start|>"},
        f"assistant\n{assistant_output}",
        {"token": "<|im_end|>"},
        "\n"
    ]

In [ ]:
formatted_data = [convert_to_token_format(row["input"], row["output"]) for _, row in df.iterrows()]

In [ ]:
with open("chat_token_format.json", "w") as f:
    json.dump(formatted_data, f, indent=2)

In [ ]:
! git clone https://github.com/google-research-datasets/Taskmaster.git


Cloning into 'Taskmaster'...
remote: Enumerating objects: 304, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 304 (delta 48), reused 50 (delta 37), pack-reused 224 (from 1)
Receiving objects: 100% (304/304), 108.41 MiB | 29.42 MiB/s, done.
Resolving deltas: 100% (143/143), done.
Updating files: 100% (143/143), done.


In [ ]:
df=pd.read_csv("/content/Taskmaster/TM-3-2020/splits/dev/dev-ab.tsv",sep='\t',header=None)

In [ ]:
df.columns = ['user', 'assistant', 'session_id']

In [ ]:
def extract_messages(row):
    session = []
    text = row['user']


    segments = re.split(r'(<[UA]>)', text)
    segments = [s for s in segments if s and s != "<C>"]

    for i in range(0, len(segments) - 1, 2):
        tag = segments[i]
        message = segments[i + 1].strip()
        role = "user" if tag == "<U>" else "assistant"

        session.extend([
            {"token": "<|im_start|>"},
            f"{role}\n{message}",
            {"token": "<|im_end|>"},
            "\n"
        ])

    if pd.notna(row['assistant']):
        last_msg = re.sub(r"^\[A\]", "", row["assistant"]).strip()
        session.extend([
            {"token": "<|im_start|>"},
            f"assistant\n{last_msg}",
            {"token": "<|im_end|>"},
            "\n"
        ])

    return {
        "session_id": row["session_id"],
        "messages": session
    }

sessions = df.apply(extract_messages, axis=1).tolist()


In [ ]:
with open("movie2_chat_sessions.jsonl", "w") as f:
    for session in sessions:
        f.write(json.dumps(session) + "\n")

##new data prepare

In [ ]:
import pandas as pd
df=pd.read_json("/content/Taskmaster/TM-2-2020/data/sports.json")

In [ ]:
df.head(5)

,conversation_id,instruction_id,utterances
0,dlg-0004d9ce-d9c4-4c20-a744-e25c053ea9b8,mlb-6,"[{'index': 0, 'speaker': 'USER', 'text': 'Hi. ..."
1,dlg-000ff7e6-03d2-49f2-bc35-7facd64c3a39,mlb-7,"[{'index': 0, 'speaker': 'USER', 'text': 'What..."
2,dlg-0011cca8-a3e2-4155-b1a5-c2a93c84e3d1,epl-1,"[{'index': 0, 'speaker': 'USER', 'text': 'I'm ..."
3,dlg-0032fb90-18ce-45cd-a360-ab4b139a5d29,mls-6,"[{'index': 0, 'speaker': 'ASSISTANT', 'text': ..."
4,dlg-004ef8ca-ef77-4564-980c-3705c3c681fe,mls-1,"[{'index': 0, 'speaker': 'USER', 'text': 'Hey,..."


In [ ]:
df.shape

(3481, 3)

In [ ]:
df['utterances'][0]

[{'index': 0,
  'speaker': 'USER',
  'text': "Hi. My favorite baseball team is St. Louis Cardinals and I want to ask how they're doing currently.",
  'segments': [{'start_index': 33,
    'end_index': 52,
    'text': 'St. Louis Cardinals',
    'annotations': [{'name': 'mlb.name.team'}]}]},
 {'index': 1, 'speaker': 'ASSISTANT', 'text': 'Sure.'},
 {'index': 2, 'speaker': 'USER', 'text': 'Who are they playing today?'},
 {'index': 3, 'speaker': 'ASSISTANT', 'text': 'Hold on for a moment.'},
 {'index': 4,
  'speaker': 'ASSISTANT',
  'text': "It's in the 3rd place in National League Central.",
  'segments': [{'start_index': 12,
    'end_index': 21,
    'text': '3rd place',
    'annotations': [{'name': 'mlb.place.team'}]}]},
 {'index': 5,
  'speaker': 'ASSISTANT',
  'text': 'There is no game today.',
  'segments': [{'start_index': 17,
    'end_index': 22,
    'text': 'today',
    'annotations': [{'name': 'mlb.day.match'}]}]},
 {'index': 6,
  'speaker': 'USER',
  'text': 'Did they played last S

In [ ]:
def convert_to_llama_chat_format(utterances, system_prompt=None):
    tokens = []
    if system_prompt:
        tokens.extend([
            {"token": "<|im_start|>"},
            f"system\n{system_prompt}",
            {"token": "<|im_end|>"}, "\n"
        ])

    for utt in utterances:
        speaker = utt['speaker'].strip().lower()
        if speaker not in ['user', 'assistant']:
            continue
        text = utt['text'].strip()
        tokens.extend([
            {"token": "<|im_start|>"},
            f"{speaker}\n{text}",
            {"token": "<|im_end|>"}, "\n"
        ])

    return tokens

In [ ]:
SYSTEM_PROMPT = (
    "You are a large language model (conversational chat model) trained by Kuldeep. "
    "Answer as concisely as possible.\nKnowledge cutoff: 2021-09-01\nCurrent date: 2023-03-01"
)

In [ ]:
chat_data_all_sessions = []

In [ ]:
for _, row in df.iterrows():
    session_id = row['conversation_id']
    utterances = row['utterances']

    formatted_session = {
        "session_id": session_id,
        "conversation": convert_to_llama_chat_format(utterances, system_prompt=SYSTEM_PROMPT)
    }
    chat_data_all_sessions.append(formatted_session)


In [ ]:
chat_data_all_sessions[0]

{'session_id': 'dlg-0004d9ce-d9c4-4c20-a744-e25c053ea9b8',
 'conversation': [{'token': '<|im_start|>'},
  'system\nYou are a large language model (conversational chat model) trained by Kuldeep. Answer as concisely as possible.\nKnowledge cutoff: 2021-09-01\nCurrent date: 2023-03-01',
  {'token': '<|im_end|>'},
  '\n',
  {'token': '<|im_start|>'},
  "user\nHi. My favorite baseball team is St. Louis Cardinals and I want to ask how they're doing currently.",
  {'token': '<|im_end|>'},
  '\n',
  {'token': '<|im_start|>'},
  'assistant\nSure.',
  {'token': '<|im_end|>'},
  '\n',
  {'token': '<|im_start|>'},
  'user\nWho are they playing today?',
  {'token': '<|im_end|>'},
  '\n',
  {'token': '<|im_start|>'},
  'assistant\nHold on for a moment.',
  {'token': '<|im_end|>'},
  '\n',
  {'token': '<|im_start|>'},
  "assistant\nIt's in the 3rd place in National League Central.",
  {'token': '<|im_end|>'},
  '\n',
  {'token': '<|im_start|>'},
  'assistant\nThere is no game today.',
  {'token': '<|

In [ ]:
len(chat_data_all_sessions)

3481

In [ ]:
import json
with open("sports_chat_data.json", "w") as f:
    json.dump(chat_data_all_sessions, f, indent=2)

In [ ]:
import json

def convert_to_chatformat(input_path, output_path, system_prompt=None):
    """
    input_path: path to your present JSON data (as described)
    output_path: desired .jsonl output path
    system_prompt: optional string for a system message, otherwise no system message
    """
    with open(input_path, "r", encoding="utf-8") as fin:
        conversations = json.load(fin)

    with open(output_path, "w", encoding="utf-8") as fout:
        for conv in conversations:
            messages = []
            if system_prompt:
                messages.append({"role": "system", "content": system_prompt})

            for utt in conv["utterances"]:
                speaker = utt["speaker"].strip().lower()
                if speaker == "assistant":
                    role = "assistant"
                elif speaker == "user":
                    role = "user"
                else:
                    continue  # skip unknown roles

                messages.append({
                    "role": role,
                    "content": utt["text"].strip()
                })

            if len(messages) > 1:  # avoid writing empty or system-only dialogs
                fout.write(json.dumps({"messages": messages}, ensure_ascii=False) + "\n")

# === How to use ===

convert_to_chatformat(
    input_path="/content/Taskmaster/TM-2-2020/data/movies.json",
    output_path="chat_ready_data.jsonl",
    system_prompt="You are a helpful assistant."  # Or leave out for no system prompt
)
